In [5]:
import numpy as np
import pandas as pd
import urllib

In [23]:
TITLES = ['firefighter', 'substitute teacher', 'dental hygienist', 'teacher', 'licensed practical nurse',
          'dental assistant', 'nursing assistant', 'surgical technician']
TITLE = TITLES[0]

df_paa = pd.read_csv('../output/paa_{}.csv'.format(TITLE.replace(" ", "_")))
df_paa.columns = ['orig_query', 'query', 'rank', 'question', 'answer', 'link']
print(df_paa.shape)
df_paa = df_paa.dropna(subset=['query', 'question', 'answer'])
print(df_paa.shape)
df_paa = df_paa[df_paa.apply(lambda x: "{" not in x['query'], axis=1)]
print(df_paa.shape)

(3676, 6)
(3676, 6)
(3676, 6)


In [25]:
df_paa['rank'] = (np.max(df_paa['rank'])-df_paa['rank'])/(np.max(df_paa['rank'])-np.min(df_paa['rank']))
df_group = df_paa.groupby(['question', 'link']).agg({'answer':set, 'query':set, 'rank':[sum,"count"]}).reset_index()
df_group.columns = ['question', 'link','answer', 'query_set', 'rank_sum', 'appear_count']
df_group = df_group.sort_values(by=['appear_count', 'rank_sum'], ascending=False).reset_index(drop=True)
pd.set_option('display.max_colwidth', 200)
df_group.head(2)

,question,link,answer,query_set,rank_sum,appear_count
0,What is the highest paying state for firefighters?,https://www.forbes.com/sites/andrewdepietro/2019/03/31/firefighters-salary-state/,"{10 States Where Firefighters Earn the Most Money Rank State 2017 Mean Annual Wage 1 New Jersey $75,880 2 California $73,860 3 New York $70,560 4 Washington $70,300 6 more rows}","{how much does working as a firefighter pay annually, What is the monthly income for a firefighter?, average firefighter salary, What fire department pays the most?, firefighter monthly wage, fire...",86.666667,159
1,Can you live off a firefighter salary?,https://moneynation.com/firefighter-salary/,"{Even on the typical firefighter salary of $45,970 a year, it's possible to save $4.4 million. Some firefighters use pension spiking to boost their retirement income. They log high overtime hours ...","{how much firefighter make, how much firefighter make a week, how much can a firefighter make, how much does a firefighter earn yearly, average firefighter salary per week, how much firefighter ge...",42.333333,114


In [26]:
df_qna = df_group.copy()
df_qna['rank'] = df_qna.index
df_qna['topic'] = ""
df_qna['answer'] = df_qna.apply(lambda x: list(x['answer'])[0]+" "+str(x['link']), axis=1)
df_qna = df_qna[['topic', 'question', 'rank', 'answer']]
df_qna.head()

,topic,question,rank,answer
0,,What is the highest paying state for firefighters?,0,"10 States Where Firefighters Earn the Most Money Rank State 2017 Mean Annual Wage 1 New Jersey $75,880 2 California $73,860 3 New York $70,560 4 Washington $70,300 6 more rows https://www.forbes.c..."
1,,Can you live off a firefighter salary?,1,"Even on the typical firefighter salary of $45,970 a year, it's possible to save $4.4 million. Some firefighters use pension spiking to boost their retirement income. They log high overtime hours i..."
2,,Is firefighter a good career?,2,A firefighter is the absolute best profession. The ability to help others and have fun while doing it is priceless. Working in a great station with a great crew cannot be beat. Delivering a servic...
3,,Is it worth becoming a firefighter?,3,"You are different than every other person out there trying to become a firefighter. It is my belief that for some it is, and for others it may not be, worth pursuing firefighting. Major factors yo..."
4,,Do firefighters get paid weekly?,4,"Firefighters: Under the CBA, the firefighters work on a 21-day, 168-hour pay schedule. They receive two different types of paychecks. One type of paycheck is received bi-weekly and is intended to ..."


In [7]:
file_qna = '../output/qna_{}.csv'.format(urllib.parse.quote_plus(TITLE))
df_qna.to_csv(file_qna, index=False)

### Batch run

In [31]:
TITLES = ['firefighter', 'substitute teacher', 'dental hygienist', 'teacher', 'licensed practical nurse',
          'dental assistant', 'nursing assistant', 'surgical technician']

# TITLES = ["teacher", "project manager", "business analyst", "accountant", "administrative assistant", "product manager"]


for TITLE in TITLES[-1:]:
    df_paa = pd.read_csv('../output/paa_{}.csv'.format(TITLE.replace(" ", "_")))
    df_paa.columns = ['orig_query', 'query', 'rank', 'question', 'answer', 'link']
    df_paa = df_paa.dropna(subset=['query', 'question', 'answer'])
    df_paa = df_paa[df_paa.apply(lambda x: "{" not in x['query'], axis=1)]
    df_paa['rank'] = (np.max(df_paa['rank'])-df_paa['rank'])/(np.max(df_paa['rank'])-np.min(df_paa['rank']))
    df_group = df_paa.groupby(['question', 'link']).agg({'answer':set, 'query':set, 'rank':[sum,"count"]}).reset_index()
    df_group.columns = ['question', 'link','answer', 'query_set', 'rank_sum', 'appear_count']
    df_group = df_group.sort_values(by=['appear_count', 'rank_sum'], ascending=False).reset_index(drop=True)
    df_group.to_csv('../output/ranked_{}.csv'.format(TITLE.replace(" ", "_")), index=False)
    df_qna = df_group.copy()
    df_qna['rank'] = df_qna.index
    df_qna['topic'] = ""
    df_qna['answer'] = df_qna.apply(lambda x: list(x['answer'])[0]+" "+str(x['link']), axis=1)
    df_qna = df_qna[['topic', 'question', 'rank', 'answer']]
    file_qna = '../output/qna_{}.csv'.format(urllib.parse.quote_plus(TITLE))
    df_qna.to_csv(file_qna, index=False)